In [57]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import time
import pprint
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import re
from datetime import datetime
from selenium.common.exceptions import ElementNotVisibleException, StaleElementReferenceException, NoSuchElementException

In [106]:
title.remove("\n     Sorry, this service is temporarily unavailable\n        ")
len(title)

436

In [112]:
title = []
date =[]
section = []
text = []
urls = []
url_errors = []
errors = []


competitions = [#"premier-league","champions-league", "fa-cup",
#"european-championship", "european-championship-qualifying", "world-cup", "international-friendlies",
"womens-super-league", "womens-champions-league", "fa-womens-cup", "womens-european-championship",
#"womens-european-championships-qualifying", "womens-world-cup",
#"womens-world-cup-qualifying", "womens-international-friendlies", "shebelieves-cup", "arnold-clark-cup"
]

years = ['2022', #'2023']
months = ['12', '11', '10', '09', '08', '07', '06', '05', '04', '03', '02', '01']
for comp in competitions:
    for month in months:
        url_template = f"https://www.bbc.co.uk/sport/football/{comp}/scores-fixtures/2022-{month}"
        driver = webdriver.Chrome()
        driver.get(url_template)
        elements = driver.find_elements(By.CLASS_NAME, "sp-c-fixture__block-link")
        for num in range(len(elements)):    
            try:
                driver.get(url_template)
                elements = driver.find_elements(By.CLASS_NAME, "sp-c-fixture__block-link")
                elements[num].click()
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                title.append(soup.find('h1').text)
                section.append(soup.find(class_ = "story-info").text.split("section",1)[1].split("comments",1)[0])
                text.append(soup.find(class_="qa-story-body").text.split("Player of the match",1)[0].split("Line-upsMatch Stats", 1)[0])
                date.append(soup.find("time").text)
                urls.append(driver.current_url)
            except StaleElementReferenceException:
                url_errors.append(url_template)
                errors.append(num)
            except AttributeError:
                pass

In [115]:
print(errors)
print(url_errors)
print(len(title))
print(len(date))
print(len(section))
print(len(text))
print(len(urls))

[1, 1]
['https://www.bbc.co.uk/sport/football/international-friendlies/scores-fixtures/2022-11', 'https://www.bbc.co.uk/sport/football/international-friendlies/scores-fixtures/2022-09']
26
26
26
26
26


In [116]:
data = {"title" : title,
        "date": date,
        'section' : section,
        'text': text,
       'url' : urls}

In [117]:
rawdf = pd.DataFrame(data)
rawdf

,title,date,section,text,url
0,Malta 0-1 Republic of Ireland: Belfast-born Ma...,SUN 20 Nov 2022,Republic of Ireland,Mark Sykes was introduced by Stephen Kenny in ...,https://www.bbc.co.uk/sport/football/63624383
1,Turkey 2-1 Scotland: John McGinn's goal mere c...,WED 16 Nov 2022,Football,Turkey game 'showed depth of Scotland squad' -...,https://www.bbc.co.uk/sport/football/63564471
2,Argentina: Lionel Messi scores twice against J...,WED 28 Sept 2022,Football,Messi has scored nine goals in his last three ...,https://www.bbc.co.uk/sport/football/63058346
3,Argentina 3-0 Honduras: Messi scores twice as ...,SAT 24 Sept 2022,Football,Messi is Argentina's all-time leading goalscor...,https://www.bbc.co.uk/sport/football/63019162
4,Brazil 3-0 Ghana: Tottenham forward Richarliso...,FRI 23 Sept 2022,Football,Richarlison has now scored 16 goals in 37 game...,https://www.bbc.co.uk/sport/football/63016559
5,Argentina 5-0 Estonia: Lionel Messi scores fiv...,SUN 05 Jun 2022,Football,Lionel Messi moved up to fourth in the all-tim...,https://www.bbc.co.uk/sport/football/61700518
6,Christian Eriksen scores in Denmark's 3-0 win ...,TUE 29 Mar 2022,Football,Christian Eriksen's goal was his 38th for Denm...,https://www.bbc.co.uk/sport/football/60901275
7,Norway v Armenia,TUE 29 Mar 2022,Football,,https://www.bbc.co.uk/sport/football/60921657
8,Austria 2-2 Scotland: Steve Clarke says his si...,TUE 29 Mar 2022,Football,Club Bruges centre-back Jack Hendry bundled in...,https://www.bbc.co.uk/sport/football/60874723
9,Belgium 3-0 Burkina Faso: Trossard and Benteke...,TUE 29 Mar 2022,Football,Benteke scored his 18th international goal for...,https://www.bbc.co.uk/sport/football/60921664


In [118]:
df = pd.concat([df,rawdf], ignore_index = True)
df

,title,date,section,text,url
0,Wolves 0-1 Man Utd: Marcus Rashford comes off ...,SAT 31 Dec 2022,Premier League,Marcus Rashford turned from villain to hero as...,https://www.bbc.co.uk/sport/football/64083698
1,Bournemouth 0-2 Crystal Palace: Gary O'Neil la...,SAT 31 Dec 2022,Premier League,Bournemouth have conceded more first-half goal...,https://www.bbc.co.uk/sport/football/64083697
2,Fulham 2-1 Southampton: Saints stay bottom aft...,SAT 31 Dec 2022,Premier League,Palhinha's winner is his third Fulham goal sin...,https://www.bbc.co.uk/sport/football/64083700
3,Manchester City 1-1 Everton: Demarai Gray canc...,SAT 31 Dec 2022,Premier League,Haaland's goal was his 21st of the Premier Lea...,https://www.bbc.co.uk/sport/football/64083701
4,Newcastle United 0-0 Leeds United: Magpies hel...,SAT 31 Dec 2022,Premier League,Chris Wood was denied a goal against his forme...,https://www.bbc.co.uk/sport/football/64083702
...,...,...,...,...,...
457,Germany 2-0 Israel: Goals from Chelsea duo Kai...,SAT 26 Mar 2022,European Football,Germany's last defeat came against England at ...,https://www.bbc.co.uk/sport/football/60889494
458,Netherlands 4-2 Denmark: Christian Eriksen sco...,SAT 26 Mar 2022,European Football,Christian Eriksen scored his 37th internationa...,https://www.bbc.co.uk/sport/football/60889501
459,International friendly: Northern Ireland hit l...,FRI 25 Mar 2022,Northern Ireland,Northern Ireland manager Ian BaracloughCaptain...,https://www.bbc.co.uk/sport/football/60789839
460,France 2-1 Ivory Coast: Olivier Giroud closes ...,FRI 25 Mar 2022,Football,Olivier Giroud scored his 47th goal for France...,https://www.bbc.co.uk/sport/football/60882155


In [119]:
df.section.unique()

array([' Premier League', ' European Football', ' Champions League',
       ' FA Cup', ' Football', ' Republic of Ireland',
       ' Northern Ireland'], dtype=object)

In [121]:
df.to_csv("bbc_selenium_scrape.csv")